In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import linalg

In [2]:
# List to store selected points
selected_points = []

# Mouse callback function to capture points
def select_points(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Left click
        selected_points.append((x, y))
        print(f"Point selected: {x}, {y}")
        if len(selected_points) == 4:
            cv2.destroyAllWindows()  # Close the window once 4 points are selected

def superimpose_flag(arch_image, flag_image_path, points):
    # Load the flag image
    flag_image = cv2.imread(flag_image_path)

    # Dimensions of the flag image
    h_flag, w_flag = flag_image.shape[:2]

    # Define the source points (corners of the flag)
    src_pts = np.array([[0, 0], [w_flag, 0], [w_flag, h_flag], [0, h_flag]], dtype=np.float32)

    # Define the destination points (selected points on the architectural image)
    dst_pts = np.array(points, dtype=np.float32)

    # Compute the homography matrix
    H, _ = cv2.findHomography(src_pts, dst_pts)

    # Warp the flag image to fit the planar surface in the architectural image
    warped_flag = cv2.warpPerspective(flag_image, H, (arch_image.shape[1], arch_image.shape[0]))

    # Create a mask of the warped flag (for blending)
    mask = np.zeros_like(arch_image, dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(dst_pts), (255, 255, 255))

    # Invert the mask for the background
    inverted_mask = cv2.bitwise_not(mask)

    # Blend the images: First remove the region from the architectural image, then add the warped flag
    arch_image_bg = cv2.bitwise_and(arch_image, inverted_mask)
    blended_image = cv2.add(arch_image_bg, warped_flag)

    # Return or display the final blended image
    return blended_image

# Example usage:
# flag_image_path = 'Images/flag.png'
# arch_image_path = 'Images/005.jpg'


# Example usage:
# flag_image_path = 'Images/download.png'
# arch_image_path = 'Images/Highway.jpg'

# Example usage:
flag_image_path = 'Images/cocacola.jpg'
arch_image_path = 'Images/tower.jpeg'


# Load the architectural image
arch_image = cv2.imread(arch_image_path)

# 

# Display the image and select points manually
cv2.imshow('Select 4 Points on the Architectural Image', arch_image)
cv2.setMouseCallback('Select 4 Points on the Architectural Image', select_points)

# Wait until 4 points are selected
cv2.waitKey(0)

# Check if 4 points were selected
if len(selected_points) != 4:
    print("Error: You must select exactly 4 points.")
    exit()

# Superimpose the flag with the selected points
result_image = superimpose_flag(arch_image, flag_image_path, selected_points)

# Display the result
cv2.imshow('Superimposed Flag', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Point selected: 229, 207
Point selected: 393, 76
Point selected: 521, 166
Point selected: 377, 414
